# Final Project | Stage 2
## Evaluating and Comparing Different Machine Learning Models Performance on Autoimmune Disease Prediction


### Imports, Global Settings, and Reproducibility

In [9]:
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Set random seed to ensure reproducible results across multiple runs
# This is critical for scientific validity - same seed = same results
RANDOM_STATE = 42
random.seed(RANDOM_STATE)           # Python's random module
np.random.seed(RANDOM_STATE)        # NumPy's random number generator
torch.manual_seed(RANDOM_STATE)     # PyTorch's random number generator


### Data Import and Preprocessing
This section:
- Loads the dataset
- Encodes categorical variables
- Scales numerical features
- Splits data into training and testing sets

In [10]:
# Load dataset
data = pd.read_csv(
"/Users/Caitlynrose/Machine_Learning/Final_Project/health_data/Final_Balanced_Autoimmune_Disorder_Dataset.csv"
)

# Encode Diagnosis labels (e.g., "Rheumatoid arthritis"=0, "SLE"=1, etc.)
# We save the encoder so we can convert predictions back to disease names later
diagnosis_encoder = LabelEncoder()
data["Gender"] = diagnosis_encoder.fit_transform(data["Gender"])
data["Diagnosis"] = diagnosis_encoder.fit_transform(data["Diagnosis"])
diagnosis_labels = diagnosis_encoder.classes_  # Save original disease names
num_classes = len(diagnosis_labels)            # Number of different diseases

# X = features (input variables used to make predictions)
# y = target (the diagnosis we're trying to predict)
X = data.drop(columns=["Patient_ID", "Diagnosis"])  # Remove ID and target
y = data["Diagnosis"]                                # Keep only target

# Standardize features by removing mean and scaling to unit variance
# This is critical for:
# 1. Logistic Regression - sensitive to feature scales
# 2. Neural Networks - helps with convergence and training stability
# Random Forest doesn't need scaling but it doesn't hurt
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training (70%) and testing (30%) sets
# - Training set: used to train models and tune hyperparameters
# - Testing set: held out to evaluate final performance (unseen data)
# stratify=y ensures both sets have same proportion of each disease
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=RANDOM_STATE, stratify=y
)

print(f"\n✓ Preprocessing complete")
print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")
print(f"Number of features: {X_train.shape[1]}\n")




✓ Preprocessing complete
Training samples: 8198
Testing samples: 3514
Number of features: 77



In [11]:
# ============================================================================
# HELPER FUNCTION FOR RESULTS DISPLAY
# ============================================================================
def print_model_results(model_name, best_params, train_acc, test_acc, f1, 
                        y_true, y_pred, labels, num_samples=10):

    print("=" * 80)
    print(f"{model_name.upper()} - RESULTS")
    print("=" * 80)
    
    # Display all hyperparameters that were optimized
    print("\n📊 Best Hyperparameters:")
    for param, value in best_params.items():
        print(f"   • {param}: {value}")
    
    # Display performance metrics
    # Training accuracy: how well the model memorizes training data
    # Testing accuracy: how well the model generalizes
    # F1 score: balances precision and recall
    print("\n📈 Performance Metrics:")
    print(f"   • Training Accuracy:    {train_acc:.4f} ({train_acc*100:.2f}%)")
    print(f"   • Testing Accuracy:     {test_acc:.4f} ({test_acc*100:.2f}%)")
    print(f"   • F1 Score (Weighted):  {f1:.4f}")
    
    # Show sample predictions to give intuition about model performance
    # This helps us see what kinds of mistakes the model is making
    print(f"\n🔍 Sample Predictions (First {num_samples}):")
    print(f"   {'Predicted':<40} {'Actual':<40} {'Match'}")
    print(f"   {'-'*40} {'-'*40} {'-'*5}")
    
    matches = 0
    for i in range(min(num_samples, len(y_pred))):
        # Convert numerical predictions back to disease names
        pred_label = labels[y_pred[i]]
        true_label = labels[y_true.iloc[i] if hasattr(y_true, 'iloc') else y_true[i]]
        match = "✓" if pred_label == true_label else "✗"
        if pred_label == true_label:
            matches += 1
        print(f"   {pred_label:<40} {true_label:<40} {match}")
    
    print(f"\n   Sample accuracy: {matches}/{num_samples}")
    print()


### Logistic Regression

In [12]:
# ============================================================================
# MODEL 1: LOGISTIC REGRESSION
# ============================================================================
print("=" * 80)
print("MODEL 1: LOGISTIC REGRESSION")
print("=" * 80)
print("\n🔍 Starting comprehensive grid search...\n")

# ============================================================================
# LOGISTIC REGRESSION HYPERPARAMETER GRID
# ============================================================================
# We test different combinations of hyperparameters to find the best model
param_grid_lr = {
    # C: Inverse of regularization strength (smaller = more regularization)
    # Regularization prevents overfitting by penalizing large coefficients
    "C": [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    
    # solver: Algorithm used to optimize the model
    # Different solvers work better for different data sizes/types
    "solver": ["lbfgs", "liblinear", "saga"],
    
    # penalty: Type of regularization to apply
    # L2 (ridge) penalty helps prevent overfitting
    "penalty": ["l2"],
    
    # max_iter: Maximum iterations for solver to converge
    # More iterations = more time but potentially better solution
    "max_iter": [1000, 2000, 3000]
}

# ============================================================================
# GRID SEARCH WITH CROSS-VALIDATION
# ============================================================================
# GridSearchCV tests every combination of hyperparameters
# cv=5 means 5-fold cross-validation: split training data into 5 parts,
# train on 4, validate on 1, repeat 5 times, average results
# This gives us a robust estimate of how well each configuration performs
grid_lr = GridSearchCV(
    estimator=LogisticRegression(random_state=RANDOM_STATE),
    param_grid=param_grid_lr,
    scoring="accuracy",      # Optimize for accuracy
    cv=5,                    # 5-fold cross-validation
    n_jobs=-1,              # Use all CPU cores for parallel processing
    verbose=1               # Print progress
)

# Train the model and find best hyperparameters
grid_lr.fit(X_train, y_train)
best_lr = grid_lr.best_estimator_  # Get the best model

# ============================================================================
# EVALUATE LOGISTIC REGRESSION
# ============================================================================
# Generate predictions on both training and testing sets
lr_train_preds = best_lr.predict(X_train)
lr_test_preds = best_lr.predict(X_test)

# Calculate performance metrics
lr_train_acc = accuracy_score(y_train, lr_train_preds)  # Training accuracy
lr_test_acc = accuracy_score(y_test, lr_test_preds)      # Testing accuracy (most important!)
lr_f1 = f1_score(y_test, lr_test_preds, average='weighted')  # F1 score

print("\n✓ Logistic Regression training complete\n")

# Display all results in a formatted way
print_model_results(
    "Logistic Regression",
    grid_lr.best_params_,
    lr_train_acc,
    lr_test_acc,
    lr_f1,
    y_test,
    lr_test_preds,
    diagnosis_labels
)

MODEL 1: LOGISTIC REGRESSION

🔍 Starting comprehensive grid search...

Fitting 5 folds for each of 63 candidates, totalling 315 fits


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



✓ Logistic Regression training complete

LOGISTIC REGRESSION - RESULTS

📊 Best Hyperparameters:
   • C: 0.001
   • max_iter: 1000
   • penalty: l2
   • solver: lbfgs

📈 Performance Metrics:
   • Training Accuracy:    0.4525 (45.25%)
   • Testing Accuracy:     0.4254 (42.54%)
   • F1 Score (Weighted):  0.4236

🔍 Sample Predictions (First 10):
   Predicted                                Actual                                   Match
   ---------------------------------------- ---------------------------------------- -----
   Graves' disease                          Sjögren syndrome                         ✗
   Systemic lupus erythematosus (SLE)       Autoimmune orchitis                      ✗
   Systemic lupus erythematosus (SLE)       Systemic lupus erythematosus (SLE)       ✓
   Sjögren syndrome                         Sjögren syndrome                         ✓
   Graves' disease                          Graves' disease                          ✓
   Autoimmune orchitis                

### Random Forest

In [14]:
# ============================================================================
# MODEL 2: RANDOM FOREST
# ============================================================================
print("=" * 80)
print("MODEL 2: RANDOM FOREST")
print("=" * 80)
print("\n🌲 Starting condensed grid search...\n")

# ============================================================================
# RANDOM FOREST HYPERPARAMETER GRID (CONDENSED)
# ============================================================================
# Reduce the number of hyperparameter combinations to speed up the search
param_grid_rf = {
    "n_estimators": [100, 200],  
    "max_depth": [None, 20, 40],  
    "min_samples_split": [2, 10], 
    "min_samples_leaf": [1, 4],  
    "max_features": ["sqrt", "log2"],  
    "bootstrap": [True] 
}

# ============================================================================
# GRID SEARCH WITH CROSS-VALIDATION
# ============================================================================
grid_rf = GridSearchCV(
    estimator=RandomForestClassifier(random_state=RANDOM_STATE),
    param_grid=param_grid_rf,
    scoring="accuracy",
    cv=3,  # Reduce cross-validation folds to 3
    n_jobs=-1,
    verbose=1
)

# Train and find best hyperparameters
grid_rf.fit(X_train, y_train)
best_rf = grid_rf.best_estimator_

# ============================================================================
# EVALUATE RANDOM FOREST
# ============================================================================
rf_train_preds = best_rf.predict(X_train)
rf_test_preds = best_rf.predict(X_test)

rf_train_acc = accuracy_score(y_train, rf_train_preds)
rf_test_acc = accuracy_score(y_test, rf_test_preds)
rf_f1 = f1_score(y_test, rf_test_preds, average='weighted')

print("\n✓ Random Forest training complete\n")

print_model_results(
    "Random Forest",
    grid_rf.best_params_,
    rf_train_acc,
    rf_test_acc,
    rf_f1,
    y_test,
    rf_test_preds,
    diagnosis_labels
)


MODEL 2: RANDOM FOREST

🌲 Starting condensed grid search...

Fitting 3 folds for each of 48 candidates, totalling 144 fits

✓ Random Forest training complete

RANDOM FOREST - RESULTS

📊 Best Hyperparameters:
   • bootstrap: True
   • max_depth: None
   • max_features: sqrt
   • min_samples_leaf: 4
   • min_samples_split: 2
   • n_estimators: 100

📈 Performance Metrics:
   • Training Accuracy:    0.9790 (97.90%)
   • Testing Accuracy:     0.9772 (97.72%)
   • F1 Score (Weighted):  0.9732

🔍 Sample Predictions (First 10):
   Predicted                                Actual                                   Match
   ---------------------------------------- ---------------------------------------- -----
   Sjögren syndrome                         Sjögren syndrome                         ✓
   Autoimmune orchitis                      Autoimmune orchitis                      ✓
   Systemic lupus erythematosus (SLE)       Systemic lupus erythematosus (SLE)       ✓
   Sjögren syndrome            

### Neural Network 

In [15]:
# ============================================================================
# MODEL 3: COMPLEX NEURAL NETWORK WITH REGULARIZATION
# ============================================================================
print("=" * 80)
print("MODEL 3: NEURAL NETWORK (COMPLEX + REGULARIZATION)")
print("=" * 80)
print("\nStrategy: Start with complex architecture, tune regularization\n")

# ============================================================================
# NEURAL NETWORK CLASS DEFINITION
# ============================================================================
class ComplexAutoimmuneNet(nn.Module):
    """
    Complex neural network with multiple hidden layers and dropout regularization
    
    Professor's guidance: Make the network complex, then control overfitting
    through regularization (dropout + L2 weight decay)
    
    Architecture:
    - Multiple hidden layers with ReLU activation
    - Dropout after each hidden layer to prevent overfitting
    - Final output layer for classification
    """
    def __init__(self, input_dim, hidden_layers, dropout, num_classes):
        """
        Parameters:
        -----------
        input_dim : int
            Number of input features
        hidden_layers : list of int
            List specifying neurons in each hidden layer (e.g., [512, 256, 128])
        dropout : float
            Dropout probability (0.0 to 1.0) - higher = more regularization
        num_classes : int
            Number of output classes (diseases to predict)
        """
        super().__init__()
        layers = []
        prev_dim = input_dim
        
        # Build the network layer by layer
        for h in hidden_layers:
            # Linear transformation: y = Wx + b
            layers.append(nn.Linear(prev_dim, h))
            
            # ReLU activation: adds non-linearity, allows network to learn complex patterns
            # ReLU(x) = max(0, x) - simple but effective
            layers.append(nn.ReLU())
            
            # Dropout: randomly sets neurons to 0 during training
            # This prevents co-adaptation and forces redundancy in the network
            # Key regularization technique to prevent overfitting
            layers.append(nn.Dropout(dropout))
            
            prev_dim = h
        
        # Output layer: maps final hidden layer to class predictions
        # No activation here - we'll use CrossEntropyLoss which includes softmax
        layers.append(nn.Linear(prev_dim, num_classes))
        
        # Combine all layers into a sequential model
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        """Forward pass: compute predictions from inputs"""
        return self.model(x)

# ============================================================================
# NEURAL NETWORK TRAINING FUNCTION
# ============================================================================
def train_and_eval_nn(X_train, X_val, y_train, y_val, hidden_layers, 
                       dropout, lr, batch_size, weight_decay, epochs=200):
    """
    Train and evaluate neural network with given hyperparameters
    
    This function:
    1. Converts data to PyTorch tensors
    2. Initializes the network
    3. Trains using mini-batch gradient descent
    4. Evaluates on validation set
    
    Parameters:
    -----------
    X_train, X_val : array
        Training and validation features
    y_train, y_val : array
        Training and validation labels
    hidden_layers : list
        Network architecture
    dropout : float
        Dropout probability for regularization
    lr : float
        Learning rate for optimizer
    batch_size : int
        Number of samples per gradient update
    weight_decay : float
        L2 regularization strength (applied to weights in optimizer)
    epochs : int
        Number of complete passes through training data
    
    Returns:
    --------
    model : trained neural network
    train_acc : training accuracy
    val_acc : validation accuracy
    """
    
    # ========================================================================
    # CONVERT DATA TO PYTORCH TENSORS
    # ========================================================================
    # PyTorch requires data in tensor format (similar to NumPy arrays)
    X_train_t = torch.tensor(X_train, dtype=torch.float32)
    X_val_t = torch.tensor(X_val, dtype=torch.float32)
    y_train_t = torch.tensor(y_train.values if hasattr(y_train, 'values') else y_train, dtype=torch.long)
    y_val_t = torch.tensor(y_val.values if hasattr(y_val, 'values') else y_val, dtype=torch.long)
    
    # ========================================================================
    # INITIALIZE MODEL AND TRAINING COMPONENTS
    # ========================================================================
    model = ComplexAutoimmuneNet(X_train.shape[1], hidden_layers, dropout, num_classes)
    
    # Adam optimizer: adaptive learning rate method
    # weight_decay: L2 regularization penalty on weights (prevents large weights)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    # CrossEntropyLoss: standard loss for multi-class classification
    # Combines LogSoftmax and NLLLoss - computes how wrong predictions are
    criterion = nn.CrossEntropyLoss()
    
    # ========================================================================
    # TRAINING LOOP
    # ========================================================================
    for epoch in range(epochs):
        model.train()  # Set model to training mode (enables dropout)
        
        # Shuffle training data each epoch for better generalization
        perm = torch.randperm(X_train_t.size(0))
        
        # Mini-batch gradient descent: update weights after each batch
        # Faster and often better than using entire dataset at once
        for i in range(0, X_train_t.size(0), batch_size):
            idx = perm[i:i + batch_size]
            
            # Zero gradients from previous step
            optimizer.zero_grad()
            
            # Forward pass: compute predictions
            outputs = model(X_train_t[idx])
            
            # Compute loss: how wrong are our predictions?
            loss = criterion(outputs, y_train_t[idx])
            
            # Backward pass: compute gradients
            loss.backward()
            
            # Update weights based on gradients
            optimizer.step()
    
    # ========================================================================
    # EVALUATION
    # ========================================================================
    model.eval()  # Set to evaluation mode (disables dropout)
    
    # Don't compute gradients during evaluation (saves memory and computation)
    with torch.no_grad():
        # Get predictions: choose class with highest score
        train_preds = torch.argmax(model(X_train_t), dim=1)
        val_preds = torch.argmax(model(X_val_t), dim=1)
    
    # Calculate accuracies
    train_acc = accuracy_score(y_train_t, train_preds)
    val_acc = accuracy_score(y_val_t, val_preds)
    
    return model, train_acc, val_acc

# ============================================================================
# NEURAL NETWORK HYPERPARAMETER SEARCH
# ============================================================================
print("🧠 Starting comprehensive hyperparameter search...\n")

# Create validation split from training data
# This is separate from the test set - used only for hyperparameter tuning
# Test set remains completely untouched until final evaluation
X_train_nn, X_val_nn, y_train_nn, y_val_nn = train_test_split(
    X_train, y_train, test_size=0.2, random_state=RANDOM_STATE, stratify=y_train
)

# ============================================================================
# DEFINE SEARCH SPACE
# ============================================================================
# Following professor's advice: Start with COMPLEX architectures
# These are deep (4-5 layers) and wide (512-2048 neurons)
nn_search_space = {
    # Complex architectures - various depths and widths
    "hidden_layers": [
        [512, 256, 128, 64],           # Deep pyramid
        [1024, 512, 256, 128],         # Wider deep pyramid
        [512, 512, 256, 128, 64],      # Extra deep
        [1024, 512, 512, 256, 128],    # Extra deep and wide
        [1024, 1024, 512],             # Very wide
        [2048, 1024, 512],             # Extremely wide
        [512, 256, 256, 128],          # Repeated middle layers
        [1024, 512, 256, 256, 128],    # Complex mixed
    ],
    
    # Dropout: PRIMARY regularization method
    # Randomly drops neurons during training to prevent overfitting
    # Higher values = more regularization = less overfitting but potentially underfitting
    "dropout": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6],
    
    # Weight decay: SECONDARY regularization (L2 penalty on weights)
    # Prevents weights from growing too large
    # Works together with dropout for strong regularization
    "weight_decay": [1e-5, 1e-4, 1e-3, 1e-2],
    
    # Learning rate: controls how big each gradient descent step is
    # Too high = unstable training, too low = slow convergence
    "lr": [1e-2, 5e-3, 1e-3, 5e-4, 1e-4],
    
    # Batch size: number of samples before updating weights
    # Smaller = more updates but noisier, larger = fewer but stabler updates
    "batch_size": [16, 32, 64, 128]
}

# ============================================================================
# RANDOMIZED HYPERPARAMETER SEARCH
# ============================================================================
# We use random search instead of grid search because:
# 1. Grid search would take too long (thousands of combinations)
# 2. Random search is more efficient - can find good solutions faster
# 3. Allows us to explore diverse configurations
num_trials = 50  # Test 50 random configurations
results = []

print(f"Performing {num_trials} trials...\n")

for trial in range(num_trials):
    # Randomly sample one configuration
    config = {
        "hidden_layers": random.choice(nn_search_space["hidden_layers"]),
        "dropout": random.choice(nn_search_space["dropout"]),
        "weight_decay": random.choice(nn_search_space["weight_decay"]),
        "lr": random.choice(nn_search_space["lr"]),
        "batch_size": random.choice(nn_search_space["batch_size"])
    }
    
    # Train model with this configuration
    _, _, val_acc = train_and_eval_nn(
        X_train_nn, X_val_nn, y_train_nn, y_val_nn, **config, epochs=150
    )
    
    # Save results
    results.append({**config, "val_accuracy": val_acc})
    
    # Print progress every 10 trials
    if (trial + 1) % 10 == 0:
        best_so_far = max(results, key=lambda x: x["val_accuracy"])["val_accuracy"]
        print(f"Trial {trial + 1}/{num_trials} complete | Best val acc so far: {best_so_far:.4f}")

# Find the best configuration from all trials
best_nn_config = max(results, key=lambda x: x["val_accuracy"])

print(f"\n✓ Hyperparameter search complete!")
print(f"Best validation accuracy: {best_nn_config['val_accuracy']:.4f}")
print(f"\nBest architecture: {best_nn_config['hidden_layers']}")
print(f"Best dropout: {best_nn_config['dropout']}")
print(f"Best weight_decay: {best_nn_config['weight_decay']}\n")

# ============================================================================
# TRAIN FINAL NEURAL NETWORK
# ============================================================================
print("🚀 Training final Neural Network with best hyperparameters...\n")

# Now train on FULL training set with the best hyperparameters
# Use more epochs since this is our final model
final_model, _, _ = train_and_eval_nn(
    X_train, X_test, y_train, y_test,
    hidden_layers=best_nn_config["hidden_layers"],
    dropout=best_nn_config["dropout"],
    weight_decay=best_nn_config["weight_decay"],
    lr=best_nn_config["lr"],
    batch_size=best_nn_config["batch_size"],
    epochs=250  # More training for final model
)

# ============================================================================
# EVALUATE FINAL NEURAL NETWORK
# ============================================================================
final_model.eval()  # Set to evaluation mode
with torch.no_grad():
    X_train_t = torch.tensor(X_train, dtype=torch.float32)
    X_test_t = torch.tensor(X_test, dtype=torch.float32)
    nn_train_preds = torch.argmax(final_model(X_train_t), dim=1).numpy()
    nn_test_preds = torch.argmax(final_model(X_test_t), dim=1).numpy()

# Calculate final metrics
nn_train_acc = accuracy_score(y_train, nn_train_preds)
nn_test_acc = accuracy_score(y_test, nn_test_preds)
nn_f1 = f1_score(y_test, nn_test_preds, average='weighted')

print("✓ Final model training complete\n")

# Prepare parameters dictionary for display
best_params_nn = {
    "architecture": best_nn_config["hidden_layers"],
    "dropout": best_nn_config["dropout"],
    "weight_decay (L2)": best_nn_config["weight_decay"],
    "learning_rate": best_nn_config["lr"],
    "batch_size": best_nn_config["batch_size"],
    "epochs": 250
}

print_model_results(
    "Neural Network (Complex + Regularization)",
    best_params_nn,
    nn_train_acc,
    nn_test_acc,
    nn_f1,
    y_test,
    nn_test_preds,
    diagnosis_labels
)


MODEL 3: NEURAL NETWORK (COMPLEX + REGULARIZATION)

Strategy: Start with complex architecture, tune regularization

🧠 Starting comprehensive hyperparameter search...

Performing 50 trials...



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Trial 10/50 complete | Best val acc so far: 0.9793


KeyboardInterrupt: 

### FINAL COMPARISON AND BEST MODEL IDENTIFICATION

In [ ]:
# ============================================================================
# FINAL COMPARISON AND BEST MODEL IDENTIFICATION
# ============================================================================
print("=" * 80)
print("FINAL MODEL COMPARISON")
print("=" * 80)

# Display side-by-side comparison of all three models
print(f"\n{'Model':<35} {'Train Acc':<12} {'Test Acc':<12} {'F1 Score':<12}")
print("-" * 80)
print(f"{'Logistic Regression':<35} {lr_train_acc:<12.4f} {lr_test_acc:<12.4f} {lr_f1:<12.4f}")
print(f"{'Random Forest':<35} {rf_train_acc:<12.4f} {rf_test_acc:<12.4f} {rf_f1:<12.4f}")
print(f"{'Neural Network (Complex+Reg)':<35} {nn_train_acc:<12.4f} {nn_test_acc:<12.4f} {nn_f1:<12.4f}")
print()

# ============================================================================
# IDENTIFY THE BEST MODEL
# ============================================================================
# We select the best model based on TEST accuracy (not training!)
# Test accuracy tells us how well the model generalizes to unseen data
models = [
    ("Logistic Regression", lr_test_acc, lr_f1),
    ("Random Forest", rf_test_acc, rf_f1),
    ("Neural Network", nn_test_acc, nn_f1)
]

# Find model with highest test accuracy
best_model = max(models, key=lambda x: x[1])

# Display the winner!
print("=" * 80)
print("🏆 BEST MODEL IDENTIFIED")
print("=" * 80)
print(f"\nModel: {best_model[0]}")
print(f"Test Accuracy: {best_model[1]:.4f} ({best_model[1]*100:.2f}%)")
print(f"F1 Score: {best_model[2]:.4f}")
print("\n" + "=" * 80)


## Applying Model to My Own Data

In [ ]:
print("\n" + "=" * 80)
print("PERSONAL HEALTH DATA PREDICTION")
print("=" * 80)
print("\nApplying the best model to your personal health data...\n")


your_health_data = pd.read_csv("/Users/Caitlynrose/Machine_Learning/Final_Project/health_data/my_health_data.csv")

# ============================================================================
# PREPARE YOUR DATA FOR PREDICTION
# ============================================================================
# Convert your data dictionary to a DataFrame (same format as training data)
your_data_df = pd.DataFrame([your_health_data])

# Encode Gender the same way we did for training data
# IMPORTANT: Must use the same encoder that was fit on training data
your_data_df["Gender"] = gender_encoder.transform(your_data_df["Gender"])

# Make sure columns are in the same order as training data
# Get the feature names from the original training data (before scaling)
feature_names = X.columns if hasattr(X, 'columns') else [f"Feature_{i}" for i in range(X.shape[1])]

# Reorder your data to match training data column order
# If you're missing any features, this will error - make sure to include all!
try:
    your_data_ordered = your_data_df[feature_names]
except KeyError as e:
    print(f"\n⚠️ ERROR: Missing feature in your data: {e}")
    print(f"\nRequired features: {list(feature_names)}")
    print(f"Your features: {list(your_data_df.columns)}")
    print("\nPlease add all required features to your_health_data dictionary")
    exit()


# ============================================================================
# SCALE YOUR DATA
# ============================================================================
# CRITICAL: Must use the SAME scaler that was fit on training data
# This ensures your data is on the same scale as the training data
your_data_scaled = scaler.transform(your_data_ordered)

# ============================================================================
# MAKE PREDICTIONS WITH ALL THREE MODELS
# ============================================================================
print("🔮 Generating predictions from all three models...\n")

# Logistic Regression Prediction
lr_your_pred = best_lr.predict(your_data_scaled)[0]
lr_your_pred_proba = best_lr.predict_proba(your_data_scaled)[0]

# Random Forest Prediction
rf_your_pred = best_rf.predict(your_data_scaled)[0]
rf_your_pred_proba = best_rf.predict_proba(your_data_scaled)[0]

# Neural Network Prediction
final_model.eval()
with torch.no_grad():
    your_data_tensor = torch.tensor(your_data_scaled, dtype=torch.float32)
    nn_your_logits = final_model(your_data_tensor)
    nn_your_pred = torch.argmax(nn_your_logits, dim=1).item()
    # Calculate probabilities using softmax
    nn_your_pred_proba = torch.softmax(nn_your_logits, dim=1).numpy()[0]

# ============================================================================
# DISPLAY PREDICTIONS IN A COMPREHENSIVE FORMAT
# ============================================================================
print("=" * 80)
print("YOUR PERSONAL HEALTH PREDICTION RESULTS")
print("=" * 80)

# Display input data
print("\n📋 Your Input Data:")
print("-" * 80)
for feature, value in your_health_data.items():
    print(f"   {feature:<30} {value}")

# Display predictions from each model
print("\n" + "=" * 80)
print("🔮 PREDICTIONS FROM EACH MODEL")
print("=" * 80)

print(f"\n1️⃣ Logistic Regression:")
print(f"   Predicted Diagnosis: {diagnosis_labels[lr_your_pred]}")
print(f"   Confidence: {lr_your_pred_proba[lr_your_pred]*100:.2f}%")

print(f"\n2️⃣ Random Forest:")
print(f"   Predicted Diagnosis: {diagnosis_labels[rf_your_pred]}")
print(f"   Confidence: {rf_your_pred_proba[rf_your_pred]*100:.2f}%")

print(f"\n3️⃣ Neural Network:")
print(f"   Predicted Diagnosis: {diagnosis_labels[nn_your_pred]}")
print(f"   Confidence: {nn_your_pred_proba[nn_your_pred]*100:.2f}%")

# ============================================================================
# BEST MODEL PREDICTION (THE ONE TO TRUST MOST)
# ============================================================================
print("\n" + "=" * 80)
print(f"🏆 BEST MODEL PREDICTION ({best_model[0].upper()})")
print("=" * 80)

# Use the best performing model for the final prediction
if best_model[0] == "Logistic Regression":
    best_pred = lr_your_pred
    best_proba = lr_your_pred_proba
elif best_model[0] == "Random Forest":
    best_pred = rf_your_pred
    best_proba = rf_your_pred_proba
else:  # Neural Network
    best_pred = nn_your_pred
    best_proba = nn_your_pred_proba

print(f"\nPredicted Diagnosis: {diagnosis_labels[best_pred]}")
print(f"Confidence: {best_proba[best_pred]*100:.2f}%") # Calculated from softmax output by taking the max value from softmax output

# ============================================================================
# SHOW TOP 3 MOST LIKELY DIAGNOSES
# ============================================================================
print(f"\nTop 3 Most Likely Diagnoses:")
print(f"   {'Rank':<6} {'Diagnosis':<40} {'Probability':<12}")
print(f"   {'-'*6} {'-'*40} {'-'*12}")

# Get indices of top 3 predictions sorted by probability
top_3_indices = np.argsort(best_proba)[-3:][::-1]

for rank, idx in enumerate(top_3_indices, 1):
    diagnosis = diagnosis_labels[idx]
    probability = best_proba[idx] * 100
    marker = "PREDICTION" if idx == best_pred else ""
    print(f"   {rank:<6} {diagnosis:<40} {probability:>6.2f}%  {marker}")


print("\n✅ Personal prediction complete!")
print("=" * 80)